## Whitehat Hack Funds Disbursement
We need to disburse funds to users whose funds we rescued. The purpose of this notebook is to have a single source of truth for how many funds we owe each user.

#### Decisions / Assumptions
* The point in time we will use to calculate a users assets in the protocol will be the block before the first whitehat on each chain


In [18]:
import os
from web3 import Web3
from dotenv import load_dotenv
import requests
import json
from decimal import Decimal
import csv    

In [19]:
"""
   _   _   _   _   _   _   _   _   _  
  / \ / \ / \ / \ / \ / \ / \ / \ / \ 
 ( C | o | n | s | t | a | n | t | s )
  \_/ \_/ \_/ \_/ \_/ \_/ \_/ \_/ \_/ 
"""

####################################################
# Environment variables
####################################################
load_dotenv()
ALCHEMY_API_KEY = os.getenv('ALCHEMY_API_KEY')

####################################################
# RPC Clients
####################################################
w3_mainnet = Web3(Web3.HTTPProvider(f'https://eth-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}'))
w3_base = Web3(Web3.HTTPProvider(f'https://base-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}'))
w3_unichain = Web3(Web3.HTTPProvider(f'https://unichain-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}'))

chain_to_rpc_client = {
    1: w3_mainnet,
    8453: w3_base,
    130: w3_unichain
}

####################################################
# Whitehat block nums
####################################################
chain_to_whitehat_blocks = {
    130: 25669817,  # unichain
    8453: 34814419, # base
    1: 23242436     # mainnet
}

####################################################
# Subgraph urls
####################################################
chain_to_subgraph_url = {
    # mainnet
    1: 'https://api.goldsky.com/api/public/project_cl9gc21q105380hxuh8ks53k3/subgraphs/panoptic-subgraph-mainnet/dev/gn',
    # unichain
    130: 'https://api.goldsky.com/api/public/project_cl9gc21q105380hxuh8ks53k3/subgraphs/panoptic-subgraph-unichain/dev/gn',
    # base
    8453: 'https://api.goldsky.com/api/public/project_cl9gc21q105380hxuh8ks53k3/subgraphs/panoptic-subgraph-base/dev/gn',
}

####################################################
# Contract addrs
####################################################

chain_to_contract_addrs = {
    # mainnet
    1: {
        'SemiFungiblePositionManager': '0x0000000000000DEdEDdD16227aA3D836C5753194',
        'PanopticFactory': '0x000000000000010a1DEc6c46371A28A071F8bb01',
        'PanopticPoolTemplate': '0x0000000000001B1A7fe31692d107cAA42fb06862',
        'UniswapV3Factory': '0x1F98431c8aD98523631AE4a59f267346ea31F984',
        'NonFungiblePositionManager': '0xc36442b4a4522e871399cd717abdd847ab11fe88',
        'PanopticQuery': '0x95A5fF032C728F7F16e7fBd4F4EA2d0C0623Ffbb',
        'UniswapHelper': '0x2049aa04779e56aB54585dA0c3639f139a37A168',
        'UniswapMigrator': '0xd6f8c13ec833ed8afabcd111adecc90a765b6ed9',
        'PoolManager': '0xE03A1074c86CFeDd5C142C4F04F1a1536e203543',
        'PanopticFactoryV1_1': '0x0000000000000CF008e9bf9D01f8306029724c80',
        'SemiFungiblePositionManagerV1_1': '0x0000000000000aAbbcfCA8100a9ee78124E97B33',
        'PanopticPoolTemplateV1_1': '0x0000000000035D9945Bf4d24393828e920376bAe',
        'StateView': '0x7ffe42c4a5deea5b0fec41c94c136cf115597227',
        'UniswapHelperV1_1': '0xfc6e5122a214d06bcbecf1d0db1130b4f1f846b8',
        'PanopticQueryV1_1': '0x518dB79DCFbb300109E3Aaf3dD79f046581ABd81',
        'PanopticMath': '0x000000000001CD07e625A9e225C37BEA50b3F441',
        'PanopticMathV1_1': '0x000000000001CD07e625A9e225C37BEA50b3F441',
    },
    # unichain
    130: {
        'SemiFungiblePositionManager': '0x0000000000000DEdEDdD16227aA3D836C5753194',
        'PanopticFactory': '0x000000000000010a1DEc6c46371A28A071F8bb01',
        'PanopticPoolTemplate': '0x0000000000001B1A7fe31692d107cAA42fb06862',
        'UniswapV3Factory': '0x1F98400000000000000000000000000000000003',
        'NonFungiblePositionManager': '0x943e6e07a7E8E791dAFC44083e54041D743C46E9',
        'PanopticQuery': '0x9574ed459807e6b4f841da754164d6f6cb331b1d',
        'UniswapHelper': '0x0A7d8F96eD5d78E2B1c6211c7Ec81Ee7fD44d217',
        'UniswapMigrator': '0x0000000000000000000000000000000000000000',
        'PoolManager': '0xE03A1074c86CFeDd5C142C4F04F1a1536e203543',
        'PanopticFactoryV1_1': '0x0000000000000CF008e9bf9D01f8306029724c80',
        'SemiFungiblePositionManagerV1_1': '0x0000000000000aAbbcfCA8100a9ee78124E97B33',
        'PanopticPoolTemplateV1_1': '0x0000000000035D9945Bf4d24393828e920376bAe',
        'StateView': '0x86e8631a016f9068c3f085faf484ee3f5fdee8f2',
        'UniswapHelperV1_1': '0xdc3b61181b041985f570237b9edc237d35789b97',
        'PanopticQueryV1_1': '0x70d973b11ae0937c4e29981cdc8ca4afe2e27959',
        'PanopticMath': '0x000000000001CD07e625A9e225C37BEA50b3F441',
        'PanopticMathV1_1': '0x000000000001CD07e625A9e225C37BEA50b3F441',
    },
    # base
    8453: {
        'SemiFungiblePositionManager': '0x0000000000000DEdEDdD16227aA3D836C5753194',
        'PanopticFactory': '0x000000000000010a1DEc6c46371A28A071F8bb01',
        'PanopticPoolTemplate': '0x0000000000001B1A7fe31692d107cAA42fb06862',
        'UniswapV3Factory': '0x33128a8fC17869897dcE68Ed026d694621f6FDfD',
        'NonFungiblePositionManager': '0x03a520b32c04bf3beef7beb72e919cf822ed34f1',
        'PanopticQuery': '0x0b9b661affb5548ec32e36a0730f7ad579d18ac4',
        'UniswapHelper': '0xf28143e3356dbcaf36b52b2a5fff976fb22e55ef',
        'UniswapMigrator': '0x0000000000000000000000000000000000000000',
        'PoolManager': '0x498581ff718922c3f8e6a244956af099b2652b2b',
        'PanopticFactoryV1_1': '0x0000000000000CF008e9bf9D01f8306029724c80',
        'SemiFungiblePositionManagerV1_1': '0x0000000000000aAbbcfCA8100a9ee78124E97B33',
        'PanopticPoolTemplateV1_1': '0x0000000000035D9945Bf4d24393828e920376bAe',
        'StateView': '0xa3c0c9b65bad0b08107aa264b0f3db444b867a71',
        'UniswapHelperV1_1': '0xea59473cdd44b859bef7270a5aa8b8758c88f8b1',
        'PanopticQueryV1_1': '0x5aa79223cd973d341d29b41d803f23a88816d904',
        'PanopticMath': '0x000000000001CD07e625A9e225C37BEA50b3F441',
        'PanopticMathV1_1': '0x000000000001CD07e625A9e225C37BEA50b3F441',
    }
}

####################################################
# Token addrs
####################################################
mainnet_weth = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
mainnet_wbtc = '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599'
mainnet_usdc = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'
mainnet_tbtc = '0x18084fbA666a33d37592fA2633fD49a74DD93a88'

####################################################
# Whitehat metadata
####################################################
attacker = '0xa1F0A9d51b592ee074eD6987006976908631503B'

panoptic_safe_address = '0x82BF455e9ebd6a541EF10b683dE1edCaf05cE7A1'

####################################################
# Abis
####################################################
# Minimal ABI for balanceOf
ERC20_ABI = [
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "type": "function"
    }
]

# minimal abi for twapFilter
PANOPTIC_MATH_ABI = [
    {
        "type": "function",
        "name": "twapFilter",
        "inputs": [
                {
                    "name": "univ3pool",
                    "type": "address",
                    "internalType": "address"
                },
            {"name": "twapWindow", "type": "uint32", "internalType": "uint32"}
        ],
        "outputs": [{"name": "", "type": "int24", "internalType": "int24"}],
        "stateMutability": "view"
    }
]

PANOPTIC_QUERY_ABI = [
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "pool",
                "type": "address"
            },
            {
                "internalType": "address", 
                "name": "account",
                "type": "address"
            },
            {
                "internalType": "bool",
                "name": "includePendingPremium",
                "type": "bool"
            },
            {
                "internalType": "uint256[]",
                "name": "positionIdList",
                "type": "uint256[]"
            },
            {
                "internalType": "int24",
                "name": "atTick",
                "type": "int24"
            }
        ],
        "name": "getNetLiquidationValue",
        "outputs": [
            {
                "internalType": "int256",
                "name": "value0",
                "type": "int256"
            },
            {
                "internalType": "int256", 
                "name": "value1",
                "type": "int256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    }
]

PANOPTIC_POOL_ABI = [
    {
      "type": "constructor",
      "inputs": [
        {
          "name": "_sfpm",
          "type": "address",
          "internalType": "contract SemiFungiblePositionManager"
        },
        { "name": "_poolManager", "type": "address", "internalType": "contract IPoolManager" }
      ],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "assertMinCollateralValues",
      "inputs": [
        { "name": "minValue0", "type": "uint256", "internalType": "uint256" },
        { "name": "minValue1", "type": "uint256", "internalType": "uint256" }
      ],
      "outputs": [],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "burnOptions",
      "inputs": [
        { "name": "positionIdList", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "newPositionIdList", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "tickLimitLow", "type": "int24", "internalType": "int24" },
        { "name": "tickLimitHigh", "type": "int24", "internalType": "int24" },
        { "name": "usePremiaAsCollateral", "type": "bool", "internalType": "bool" }
      ],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "burnOptions",
      "inputs": [
        { "name": "tokenId", "type": "uint256", "internalType": "TokenId" },
        { "name": "newPositionIdList", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "tickLimitLow", "type": "int24", "internalType": "int24" },
        { "name": "tickLimitHigh", "type": "int24", "internalType": "int24" },
        { "name": "usePremiaAsCollateral", "type": "bool", "internalType": "bool" }
      ],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "collateralToken0",
      "inputs": [],
      "outputs": [{ "name": "", "type": "address", "internalType": "contract CollateralTracker" }],
      "stateMutability": "pure"
    },
    {
      "type": "function",
      "name": "collateralToken1",
      "inputs": [],
      "outputs": [{ "name": "", "type": "address", "internalType": "contract CollateralTracker" }],
      "stateMutability": "pure"
    },
    {
      "type": "function",
      "name": "forceExercise",
      "inputs": [
        { "name": "account", "type": "address", "internalType": "address" },
        { "name": "tokenId", "type": "uint256", "internalType": "TokenId" },
        { "name": "positionIdListExercisee", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "positionIdListExercisor", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "usePremiaAsCollateral", "type": "uint256", "internalType": "LeftRightUnsigned" }
      ],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "getAccumulatedFeesAndPositionsData",
      "inputs": [
        { "name": "user", "type": "address", "internalType": "address" },
        { "name": "includePendingPremium", "type": "bool", "internalType": "bool" },
        { "name": "positionIdList", "type": "uint256[]", "internalType": "TokenId[]" }
      ],
      "outputs": [
        { "name": "", "type": "uint256", "internalType": "LeftRightUnsigned" },
        { "name": "", "type": "uint256", "internalType": "LeftRightUnsigned" },
        { "name": "", "type": "uint256[2][]", "internalType": "uint256[2][]" }
      ],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "getOracleTicks",
      "inputs": [],
      "outputs": [
        { "name": "currentTick", "type": "int24", "internalType": "int24" },
        { "name": "fastOracleTick", "type": "int24", "internalType": "int24" },
        { "name": "slowOracleTick", "type": "int24", "internalType": "int24" },
        { "name": "latestObservation", "type": "int24", "internalType": "int24" },
        { "name": "medianData", "type": "uint256", "internalType": "uint256" }
      ],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "initialize",
      "inputs": [],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "isSafeMode",
      "inputs": [],
      "outputs": [{ "name": "", "type": "bool", "internalType": "bool" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "liquidate",
      "inputs": [
        { "name": "positionIdListLiquidator", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "liquidatee", "type": "address", "internalType": "address" },
        { "name": "positionIdList", "type": "uint256[]", "internalType": "TokenId[]" }
      ],
      "outputs": [],
      "stateMutability": "payable"
    },
    {
      "type": "function",
      "name": "mintOptions",
      "inputs": [
        { "name": "positionIdList", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "positionSize", "type": "uint128", "internalType": "uint128" },
        { "name": "effectiveLiquidityLimitX32", "type": "uint64", "internalType": "uint64" },
        { "name": "tickLimitLow", "type": "int24", "internalType": "int24" },
        { "name": "tickLimitHigh", "type": "int24", "internalType": "int24" },
        { "name": "usePremiaAsCollateral", "type": "bool", "internalType": "bool" }
      ],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "multicall",
      "inputs": [{ "name": "data", "type": "bytes[]", "internalType": "bytes[]" }],
      "outputs": [{ "name": "results", "type": "bytes[]", "internalType": "bytes[]" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "numberOfLegs",
      "inputs": [{ "name": "user", "type": "address", "internalType": "address" }],
      "outputs": [{ "name": "", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "onERC1155Received",
      "inputs": [
        { "name": "", "type": "address", "internalType": "address" },
        { "name": "", "type": "address", "internalType": "address" },
        { "name": "", "type": "uint256", "internalType": "uint256" },
        { "name": "", "type": "uint256", "internalType": "uint256" },
        { "name": "", "type": "bytes", "internalType": "bytes" }
      ],
      "outputs": [{ "name": "", "type": "bytes4", "internalType": "bytes4" }],
      "stateMutability": "pure"
    },
    {
      "type": "function",
      "name": "oracleContract",
      "inputs": [],
      "outputs": [
        { "name": "", "type": "address", "internalType": "contract IV3CompatibleOracle" }
      ],
      "stateMutability": "pure"
    },
    {
      "type": "function",
      "name": "pokeMedian",
      "inputs": [],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "poolKey",
      "inputs": [],
      "outputs": [
        {
          "name": "key",
          "type": "tuple",
          "internalType": "struct PoolKey",
          "components": [
            { "name": "currency0", "type": "address", "internalType": "Currency" },
            { "name": "currency1", "type": "address", "internalType": "Currency" },
            { "name": "fee", "type": "uint24", "internalType": "uint24" },
            { "name": "tickSpacing", "type": "int24", "internalType": "int24" },
            { "name": "hooks", "type": "address", "internalType": "contract IHooks" }
          ]
        }
      ],
      "stateMutability": "pure"
    },
    {
      "type": "function",
      "name": "positionData",
      "inputs": [
        { "name": "user", "type": "address", "internalType": "address" },
        { "name": "tokenId", "type": "uint256", "internalType": "TokenId" }
      ],
      "outputs": [
        { "name": "", "type": "int24", "internalType": "int24" },
        { "name": "", "type": "int24", "internalType": "int24" },
        { "name": "", "type": "int24", "internalType": "int24" },
        { "name": "", "type": "int24", "internalType": "int24" },
        { "name": "", "type": "int256", "internalType": "int256" },
        { "name": "", "type": "int256", "internalType": "int256" },
        { "name": "", "type": "uint128", "internalType": "uint128" }
      ],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "settleLongPremium",
      "inputs": [
        { "name": "positionIdList", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "owner", "type": "address", "internalType": "address" },
        { "name": "legIndex", "type": "uint256", "internalType": "uint256" },
        { "name": "usePremiaAsCollateral", "type": "bool", "internalType": "bool" }
      ],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "validateCollateralWithdrawable",
      "inputs": [
        { "name": "user", "type": "address", "internalType": "address" },
        { "name": "positionIdList", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "usePremiaAsCollateral", "type": "bool", "internalType": "bool" }
      ],
      "outputs": [],
      "stateMutability": "view"
    },
    { "type": "error", "name": "AccountInsolvent", "inputs": [] },
    { "type": "error", "name": "CastingError", "inputs": [] },
    { "type": "error", "name": "EffectiveLiquidityAboveThreshold", "inputs": [] },
    { "type": "error", "name": "InputListFail", "inputs": [] },
    { "type": "error", "name": "InvalidTick", "inputs": [] },
    {
      "type": "error",
      "name": "InvalidTokenIdParameter",
      "inputs": [{ "name": "parameterType", "type": "uint256", "internalType": "uint256" }]
    },
    { "type": "error", "name": "NoLegsExercisable", "inputs": [] },
    { "type": "error", "name": "NotALongLeg", "inputs": [] },
    { "type": "error", "name": "NotMarginCalled", "inputs": [] },
    { "type": "error", "name": "PoolAlreadyInitialized", "inputs": [] },
    { "type": "error", "name": "PositionAlreadyMinted", "inputs": [] },
    { "type": "error", "name": "StaleOracle", "inputs": [] },
    { "type": "error", "name": "TooManyLegsOpen", "inputs": [] },
    { "type": "error", "name": "UnderOverFlow", "inputs": [] }
  ]

COLLATERAL_TRACKER_ABI = [
    {
      "type": "constructor",
      "inputs": [
        { "name": "_commissionFee", "type": "uint256", "internalType": "uint256" },
        { "name": "_sellerCollateralRatio", "type": "uint256", "internalType": "uint256" },
        { "name": "_buyerCollateralRatio", "type": "uint256", "internalType": "uint256" },
        { "name": "_forceExerciseCost", "type": "int256", "internalType": "int256" },
        { "name": "_targetPoolUtilization", "type": "uint256", "internalType": "uint256" },
        { "name": "_saturatedPoolUtilization", "type": "uint256", "internalType": "uint256" },
        { "name": "_ITMSpreadFee", "type": "uint256", "internalType": "uint256" },
        { "name": "_manager", "type": "address", "internalType": "contract IPoolManager" }
      ],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "allowance",
      "inputs": [
        { "name": "owner", "type": "address", "internalType": "address" },
        { "name": "spender", "type": "address", "internalType": "address" }
      ],
      "outputs": [{ "name": "allowance", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "approve",
      "inputs": [
        { "name": "spender", "type": "address", "internalType": "address" },
        { "name": "amount", "type": "uint256", "internalType": "uint256" }
      ],
      "outputs": [{ "name": "", "type": "bool", "internalType": "bool" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "asset",
      "inputs": [],
      "outputs": [{ "name": "assetTokenAddress", "type": "address", "internalType": "address" }],
      "stateMutability": "pure"
    },
    {
      "type": "function",
      "name": "balanceOf",
      "inputs": [{ "name": "account", "type": "address", "internalType": "address" }],
      "outputs": [{ "name": "balance", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "convertToAssets",
      "inputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "outputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "convertToShares",
      "inputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "outputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "decimals",
      "inputs": [],
      "outputs": [{ "name": "", "type": "uint8", "internalType": "uint8" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "delegate",
      "inputs": [{ "name": "delegatee", "type": "address", "internalType": "address" }],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "deposit",
      "inputs": [
        { "name": "assets", "type": "uint256", "internalType": "uint256" },
        { "name": "receiver", "type": "address", "internalType": "address" }
      ],
      "outputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "payable"
    },
    {
      "type": "function",
      "name": "exercise",
      "inputs": [
        { "name": "optionOwner", "type": "address", "internalType": "address" },
        { "name": "longAmount", "type": "int128", "internalType": "int128" },
        { "name": "shortAmount", "type": "int128", "internalType": "int128" },
        { "name": "swappedAmount", "type": "int128", "internalType": "int128" },
        { "name": "realizedPremium", "type": "int128", "internalType": "int128" }
      ],
      "outputs": [{ "name": "", "type": "int128", "internalType": "int128" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "exerciseCost",
      "inputs": [
        { "name": "currentTick", "type": "int24", "internalType": "int24" },
        { "name": "oracleTick", "type": "int24", "internalType": "int24" },
        { "name": "positionId", "type": "uint256", "internalType": "TokenId" },
        { "name": "positionSize", "type": "uint128", "internalType": "uint128" },
        { "name": "longAmounts", "type": "int256", "internalType": "LeftRightSigned" }
      ],
      "outputs": [{ "name": "exerciseFees", "type": "int256", "internalType": "LeftRightSigned" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "getAccountMarginDetails",
      "inputs": [
        { "name": "user", "type": "address", "internalType": "address" },
        { "name": "atTick", "type": "int24", "internalType": "int24" },
        { "name": "positionBalanceArray", "type": "uint256[2][]", "internalType": "uint256[2][]" },
        { "name": "shortPremium", "type": "uint128", "internalType": "uint128" },
        { "name": "longPremium", "type": "uint128", "internalType": "uint128" }
      ],
      "outputs": [{ "name": "", "type": "uint256", "internalType": "LeftRightUnsigned" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "getPoolData",
      "inputs": [],
      "outputs": [
        { "name": "poolAssets", "type": "uint256", "internalType": "uint256" },
        { "name": "insideAMM", "type": "uint256", "internalType": "uint256" },
        { "name": "currentPoolUtilization", "type": "uint256", "internalType": "uint256" }
      ],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "initialize",
      "inputs": [],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "maxDeposit",
      "inputs": [{ "name": "", "type": "address", "internalType": "address" }],
      "outputs": [{ "name": "maxAssets", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "pure"
    },
    {
      "type": "function",
      "name": "maxMint",
      "inputs": [{ "name": "", "type": "address", "internalType": "address" }],
      "outputs": [{ "name": "maxShares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "maxRedeem",
      "inputs": [{ "name": "owner", "type": "address", "internalType": "address" }],
      "outputs": [{ "name": "maxShares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "maxWithdraw",
      "inputs": [{ "name": "owner", "type": "address", "internalType": "address" }],
      "outputs": [{ "name": "maxAssets", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "mint",
      "inputs": [
        { "name": "shares", "type": "uint256", "internalType": "uint256" },
        { "name": "receiver", "type": "address", "internalType": "address" }
      ],
      "outputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "payable"
    },
    {
      "type": "function",
      "name": "multicall",
      "inputs": [{ "name": "data", "type": "bytes[]", "internalType": "bytes[]" }],
      "outputs": [{ "name": "results", "type": "bytes[]", "internalType": "bytes[]" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "name",
      "inputs": [],
      "outputs": [{ "name": "", "type": "string", "internalType": "string" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "previewDeposit",
      "inputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "outputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "previewMint",
      "inputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "outputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "previewRedeem",
      "inputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "outputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "previewWithdraw",
      "inputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "outputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "redeem",
      "inputs": [
        { "name": "shares", "type": "uint256", "internalType": "uint256" },
        { "name": "receiver", "type": "address", "internalType": "address" },
        { "name": "owner", "type": "address", "internalType": "address" }
      ],
      "outputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "refund",
      "inputs": [
        { "name": "refunder", "type": "address", "internalType": "address" },
        { "name": "refundee", "type": "address", "internalType": "address" },
        { "name": "assets", "type": "int256", "internalType": "int256" }
      ],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "revoke",
      "inputs": [{ "name": "delegatee", "type": "address", "internalType": "address" }],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "settleLiquidation",
      "inputs": [
        { "name": "liquidator", "type": "address", "internalType": "address" },
        { "name": "liquidatee", "type": "address", "internalType": "address" },
        { "name": "bonus", "type": "int256", "internalType": "int256" }
      ],
      "outputs": [],
      "stateMutability": "payable"
    },
    {
      "type": "function",
      "name": "symbol",
      "inputs": [],
      "outputs": [{ "name": "", "type": "string", "internalType": "string" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "takeCommissionAddData",
      "inputs": [
        { "name": "optionOwner", "type": "address", "internalType": "address" },
        { "name": "longAmount", "type": "int128", "internalType": "int128" },
        { "name": "shortAmount", "type": "int128", "internalType": "int128" },
        { "name": "swappedAmount", "type": "int128", "internalType": "int128" },
        { "name": "isCovered", "type": "bool", "internalType": "bool" }
      ],
      "outputs": [
        { "name": "", "type": "uint32", "internalType": "uint32" },
        { "name": "", "type": "uint128", "internalType": "uint128" }
      ],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "totalAssets",
      "inputs": [],
      "outputs": [{ "name": "", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "totalSupply",
      "inputs": [],
      "outputs": [{ "name": "", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "transfer",
      "inputs": [
        { "name": "recipient", "type": "address", "internalType": "address" },
        { "name": "amount", "type": "uint256", "internalType": "uint256" }
      ],
      "outputs": [{ "name": "", "type": "bool", "internalType": "bool" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "transferFrom",
      "inputs": [
        { "name": "from", "type": "address", "internalType": "address" },
        { "name": "to", "type": "address", "internalType": "address" },
        { "name": "amount", "type": "uint256", "internalType": "uint256" }
      ],
      "outputs": [{ "name": "", "type": "bool", "internalType": "bool" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "unlockCallback",
      "inputs": [{ "name": "data", "type": "bytes", "internalType": "bytes" }],
      "outputs": [{ "name": "", "type": "bytes", "internalType": "bytes" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "withdraw",
      "inputs": [
        { "name": "assets", "type": "uint256", "internalType": "uint256" },
        { "name": "receiver", "type": "address", "internalType": "address" },
        { "name": "owner", "type": "address", "internalType": "address" }
      ],
      "outputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "withdraw",
      "inputs": [
        { "name": "assets", "type": "uint256", "internalType": "uint256" },
        { "name": "receiver", "type": "address", "internalType": "address" },
        { "name": "owner", "type": "address", "internalType": "address" },
        { "name": "positionIdList", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "usePremiaAsCollateral", "type": "bool", "internalType": "bool" }
      ],
      "outputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "nonpayable"
    },
    { "type": "error", "name": "CastingError", "inputs": [] },
    { "type": "error", "name": "CollateralTokenAlreadyInitialized", "inputs": [] },
    { "type": "error", "name": "DepositTooLarge", "inputs": [] },
    { "type": "error", "name": "ExceedsMaximumRedemption", "inputs": [] },
    { "type": "error", "name": "InvalidTick", "inputs": [] },
    { "type": "error", "name": "NotPanopticPool", "inputs": [] },
    { "type": "error", "name": "PositionCountNotZero", "inputs": [] },
    { "type": "error", "name": "TransferFailed", "inputs": [] },
    { "type": "error", "name": "UnauthorizedUniswapCallback", "inputs": [] },
    { "type": "error", "name": "UnderOverFlow", "inputs": [] }
  ]

## Mainnet

#### Fetch funds to disburse from Mainnet safe (as of Sep 2)

In [20]:
# Can confirm funds match safe mainnet balances here:
# https://app.safe.global/home?safe=eth:0x82BF455e9ebd6a541EF10b683dE1edCaf05cE7A1
mainnet_tue_sep_1_2025_block = 23275660

mainnet_weth_contract = w3_mainnet.eth.contract(address=mainnet_weth, abi=ERC20_ABI)
weth_balance = mainnet_weth_contract.functions.balanceOf(panoptic_safe_address).call(block_identifier=mainnet_tue_sep_1_2025_block)

mainnet_wbtc_contract = w3_mainnet.eth.contract(address=mainnet_wbtc, abi=ERC20_ABI)
wbtc_balance = mainnet_wbtc_contract.functions.balanceOf(panoptic_safe_address).call(block_identifier=mainnet_tue_sep_1_2025_block)

mainnet_tbtc_contract = w3_mainnet.eth.contract(address=mainnet_tbtc, abi=ERC20_ABI)
tbtc_balance = mainnet_tbtc_contract.functions.balanceOf(panoptic_safe_address).call(block_identifier=mainnet_tue_sep_1_2025_block)

mainnet_usdc_contract = w3_mainnet.eth.contract(address=mainnet_usdc, abi=ERC20_ABI)
usdc_balance = mainnet_usdc_contract.functions.balanceOf(panoptic_safe_address).call(block_identifier=mainnet_tue_sep_1_2025_block)

print("Mainnet funds to distribute")
print("WETH:", weth_balance)
print("WBTC:", wbtc_balance)
print("TBTC:", tbtc_balance)
print("USDC:", usdc_balance)

Mainnet funds to distribute
WETH: 79973292639517791570
WBTC: 16056508
TBTC: 49858198424970731
USDC: 39578598443


#### Mainnet user fund distribution before whitehat

> Block `t` is 1 block before the block where the first whitehat transaction on mainnet executed

To get the snapshot of funds each user had in the protocol at the time of the rescue, we calculate the following at block `t`, for each account with open positions or deposits in the Panoptic pool:
1. The net value of the account's options portfolio in token0 and token1, calculated by [`getNetLiquidationValue`](https://github.com/panoptic-labs/panoptic-v1-helper/blob/main/src/PanopticQuery.sol#L849) with arguments as follows:
    * pool - self explanatory
    * account - self explanatory
    * `includePendingPremium` - true
    * `positionIdList` - using open positions from subgraph at the block
    * `atTick` - using panoptic pool's TWAP tick at the block.
2. Calculate the share price at block `t` for collateral0 and collateral1 for each Panoptic pool (e.g. `share_price0 = `[`collateral0.totalAssets`](https://github.com/panoptic-labs/panoptic-v1-core/blob/df4dc38dee4fe29fd889cffaa8097dccc561e572/contracts/CollateralTracker.sol#L347-L351) `/` `collateral0.totalSupply`).
3. Fetch the user's collateral0 and collateral1 share balances at the block.
4. Convert the user's collateral balances to assets using the share price (`shares * share_price`).
5. Net resulting token0 and token1 assets to get each user's total funds at the time of the rescue.

Finally, as a sanity check, we net all user losses for each token - this should be within a 6% percent of the respective token's balance held in our safe.

In [21]:
def get_share_prices_per_panoptic_pool(panoptic_pool_accounts, block, chain_id):
    rpc_client = chain_to_rpc_client[chain_id]
    share_prices_per_panoptic_pool = {}
    for ppa in panoptic_pool_accounts:
        panoptic_pool_id = ppa['panopticPool']['id']
        if panoptic_pool_id not in share_prices_per_panoptic_pool:
            try:
                collateral0_contract = rpc_client.eth.contract(address=Web3.to_checksum_address(ppa['collateral0']['id']), abi=COLLATERAL_TRACKER_ABI)
                total_assets_0 = Decimal(collateral0_contract.functions.totalAssets().call(block_identifier=block))
                total_supply_0 = Decimal(collateral0_contract.functions.totalSupply().call(block_identifier=block))

                collateral1_contract = rpc_client.eth.contract(address=Web3.to_checksum_address(ppa['collateral1']['id']), abi=COLLATERAL_TRACKER_ABI)
                total_assets_1 = Decimal(collateral1_contract.functions.totalAssets().call(block_identifier=block))
                total_supply_1 = Decimal(collateral1_contract.functions.totalSupply().call(block_identifier=block))
            except Exception as e:
                print(f"Error getting share prices: {e}")
                print(f"collateral0 address: {ppa['collateral0']['id']}")
                print(f"collateral1 address: {ppa['collateral1']['id']}")
                print(f"chain_id: {chain_id}")
                print(f"block: {block}")
                raise

            share_prices_per_panoptic_pool[panoptic_pool_id] = [
                total_assets_0 / total_supply_0,
                total_assets_1 / total_supply_1
            ]
    return share_prices_per_panoptic_pool


def get_oracle_ticks_per_panoptic_pool(panoptic_pool_accounts, block, chain_id):
    rpc_client = chain_to_rpc_client[chain_id]
    oracle_ticks_per_panoptic_pool = {}

    for ppa in panoptic_pool_accounts:
        panoptic_pool_id = ppa['panopticPool']['id']
        if panoptic_pool_id not in oracle_ticks_per_panoptic_pool:
            panoptic_pool_address = Web3.to_checksum_address(panoptic_pool_id)
            panoptic_pool_contract = rpc_client.eth.contract(address=panoptic_pool_address, abi=PANOPTIC_POOL_ABI)

            # panics for unknown reason
            if panoptic_pool_id == '0x000002daae4a265e7e0af0917cf8efcf0376ecdf':
                # use current tick for this pool since getOracleTicks() is reverting for this pool
                oracle_ticks_per_panoptic_pool[panoptic_pool_id] = 248277
            else:
                try:
                    oracle_ticks_per_panoptic_pool[panoptic_pool_id] = panoptic_pool_contract.functions.getOracleTicks().call(block_identifier=block)
                except Exception as e:
                    print(f"Error getting oracle ticks: {e}")
                    print(f"oracle_ticks: {oracle_ticks_per_panoptic_pool}")
                    print(f"block: {block}")
                    print(f"chain_id: {chain_id}")
                    print(f"panoptic_pool_id: {panoptic_pool_id}")
                    raise

    return oracle_ticks_per_panoptic_pool


def get_twap_ticks_per_panoptic_pool(panoptic_pool_accounts, block, chain_id):
    TWAP_WINDOW = 600
    rpc_client = chain_to_rpc_client[chain_id]
    twap_ticks_per_panoptic_pool = {}
    panoptic_math_v1_contract = rpc_client.eth.contract(address=chain_to_contract_addrs[chain_id]['PanopticMath'], abi=PANOPTIC_MATH_ABI)
    panoptic_math_v1_1_contract  = rpc_client.eth.contract(address=chain_to_contract_addrs[chain_id]['PanopticMathV1_1'], abi=PANOPTIC_MATH_ABI)

    for ppa in panoptic_pool_accounts:
        panoptic_pool_id = ppa['panopticPool']['id']
        panoptic_pool_oracle = ppa['panopticPool']['oracleContract']
        is_v4_pool = ppa['panopticPool']['underlyingPool']['isV4Pool']
        if panoptic_pool_id not in twap_ticks_per_panoptic_pool:
            panoptic_math_contract = panoptic_math_v1_1_contract if is_v4_pool else panoptic_math_v1_contract
            print("getting twap tick. contract: ", panoptic_math_contract)
            print("panoptic_pool_id: ", panoptic_pool_id)
            print("panoptic_pool_oracle: ", panoptic_pool_oracle)
            twap_ticks_per_panoptic_pool[panoptic_pool_id] = panoptic_math_contract.functions.twapFilter(Web3.to_checksum_address(panoptic_pool_oracle), TWAP_WINDOW).call(block_identifier=block)
            print("***")

    return twap_ticks_per_panoptic_pool


def calculate_user_funds_at_time_of_rescue(panoptic_pool_accounts, block, chain_id):
    rpc_client = chain_to_rpc_client[chain_id]
    panoptic_query_contract = rpc_client.eth.contract(address=Web3.to_checksum_address(chain_to_contract_addrs[chain_id]['PanopticQuery']), abi=PANOPTIC_QUERY_ABI)

    share_prices_per_panoptic_pool = get_share_prices_per_panoptic_pool(panoptic_pool_accounts, block, chain_id)
    oracle_ticks_per_panoptic_pool = get_oracle_ticks_per_panoptic_pool(panoptic_pool_accounts, block, chain_id)
    # twap_ticks_per_panoptic_pool = get_twap_ticks_per_panoptic_pool(panoptic_pool_accounts, block, chain_id)

    account_funds_per_panoptic_pool = []

    for ppa in panoptic_pool_accounts:
        panoptic_pool_id = ppa['panopticPool']['id']
        account_id = ppa['account']['id']

        panoptic_pool_account_funds = {
            'panoptic_pool_id': panoptic_pool_id,
            'account_id': account_id,
            'nlv0': 0,
            'nlv1': 0,
            'assets0': 0,
            'assets1': 0
        }

        collateral0_contract = rpc_client.eth.contract(address=Web3.to_checksum_address(ppa['collateral0']['id']), abi=COLLATERAL_TRACKER_ABI)
        collateral1_contract = rpc_client.eth.contract(address=Web3.to_checksum_address(ppa['collateral1']['id']), abi=COLLATERAL_TRACKER_ABI)

        oracle_ticks_per_panoptic_pool = get_oracle_ticks_per_panoptic_pool(panoptic_pool_accounts, block, chain_id)
        [current_tick, fast_oracle_tick, slow_oracle_tick, latest_observation, median_data] = oracle_ticks_per_panoptic_pool[panoptic_pool_id]

        # twap_tick = twap_ticks_per_panoptic_pool[panoptic_pool_id]

        # Get NLV if user has positions open
        position_id_list = ppa['accountBalances']
        if len(position_id_list) == 0:
            panoptic_pool_account_funds['nlv0'] = 0
            panoptic_pool_account_funds['nlv1'] = 0
        else:
            nlv = panoptic_query_contract.functions.getNetLiquidationValue(
                Web3.to_checksum_address(panoptic_pool_id),
                Web3.to_checksum_address(account_id),
                True, # includePendingPremium
                position_id_list,
                fast_oracle_tick # atTick
                # twap_tick # atTick
            ).call(block_identifier=block)
            panoptic_pool_account_funds['nlv0'] = nlv[0]
            panoptic_pool_account_funds['nlv1'] = nlv[1]

        # Fetch user's balance of shares0 and shares1 and convert to assets using share price
        share_prices = share_prices_per_panoptic_pool[panoptic_pool_id]
        shares0_balance = collateral0_contract.balanceOf(account_id).call(block_identifier=block)
        assets0_balance = Decimal(shares0_balance) * share_prices[0]
        shares1_balance = collateral1_contract.balanceOf(account_id).call(block_identifier=block)
        assets1_balance = Decimal(shares1_balance) * share_prices[1]
        panoptic_pool_account_funds['assets0'] = assets0_balance
        panoptic_pool_account_funds['assets1'] = assets1_balance

        account_funds_per_panoptic_pool.append(panoptic_pool_account_funds)

    return account_funds_per_panoptic_pool
    

In [ ]:
t = chain_to_whitehat_blocks[1] - 1

GetPanopticPoolAccountsWithOpenPositionsOrCollateralDepositsQuery = '''
query GetPanopticPoolAccountsWithOpenPositionsOrCollateralDeposits($blockBeforeWhitehat: Int!) {
  panopticPoolAccounts(
    block: {number: $blockBeforeWhitehat}
    first: 1000
    where: {
      or: 
      [
        {collateral0Shares_gt: 0},
        {collateral1Shares_gt: 0},
        {accountBalances_: {isOpen: 1}}
      ]
    }
  ) {
    account {
      id
    }
    panopticPool {
      id
      oracleContract
      underlyingPool {
        isV4Pool
      }
    }
    collateral0 {
      id
    }
    collateral1 {
      id
    }
    accountBalances(
      first: 1000
      where: {isOpen: 1}
      orderBy: createdBlockNumber
      orderDirection: desc
    ) {
      tokenId {
        id
      }
    }
  }
}
'''

response = requests.post(
    chain_to_subgraph_url[1],
    json={
        'query': GetPanopticPoolAccountsWithOpenPositionsOrCollateralDepositsQuery,
        'variables': {'blockBeforeWhitehat': t}
    }
)

resp = response.json()
panoptic_pool_accounts = resp['data']['panopticPoolAccounts']
print('# of ppas at block t', len(panoptic_pool_accounts))
# print(panoptic_pool_accounts)


# of ppas at block t 886
[{'account': {'id': '0x000a460f9e5fc39b30976cbf3484d4826941f558'}, 'panopticPool': {'id': '0x000000000000305b8621e2475aee38ab5721d525', 'oracleContract': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8', 'underlyingPool': {'isV4Pool': None}}, 'collateral0': {'id': '0xc74dc5908e3e421004f533287c052bf0cc42ddc1'}, 'collateral1': {'id': '0x351efb333885c5351418ae0134dd54cac0b3143f'}, 'accountBalances': []}, {'account': {'id': '0x00678920b62b9b453337b5c781fe868dbb1366e5'}, 'panopticPool': {'id': '0x000000000000100921465982d28b37d2006e87fc', 'oracleContract': '0xcbcdf9626bc03e24f779434178a73a0b4bad62ed', 'underlyingPool': {'isV4Pool': None}}, 'collateral0': {'id': '0xb310cf625f519da965c587e22ff6ecb49809ed09'}, 'collateral1': {'id': '0x1f8d600a0211dd76a8c1ac6065bc0816afd118ef'}, 'accountBalances': []}, {'account': {'id': '0x00678920b62b9b453337b5c781fe868dbb1366e5'}, 'panopticPool': {'id': '0x000000000000305b8621e2475aee38ab5721d525', 'oracleContract': '0x8ad599c3a0ff1de082

In [ ]:
account_funds_per_panoptic_pool = calculate_user_funds_at_time_of_rescue(panoptic_pool_accounts, block=t, chain_id=1)

ppas [{'account': {'id': '0x000a460f9e5fc39b30976cbf3484d4826941f558'}, 'panopticPool': {'id': '0x000000000000305b8621e2475aee38ab5721d525', 'oracleContract': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8', 'underlyingPool': {'isV4Pool': None}}, 'collateral0': {'id': '0xc74dc5908e3e421004f533287c052bf0cc42ddc1'}, 'collateral1': {'id': '0x351efb333885c5351418ae0134dd54cac0b3143f'}, 'accountBalances': []}, {'account': {'id': '0x00678920b62b9b453337b5c781fe868dbb1366e5'}, 'panopticPool': {'id': '0x000000000000100921465982d28b37d2006e87fc', 'oracleContract': '0xcbcdf9626bc03e24f779434178a73a0b4bad62ed', 'underlyingPool': {'isV4Pool': None}}, 'collateral0': {'id': '0xb310cf625f519da965c587e22ff6ecb49809ed09'}, 'collateral1': {'id': '0x1f8d600a0211dd76a8c1ac6065bc0816afd118ef'}, 'accountBalances': []}, {'account': {'id': '0x00678920b62b9b453337b5c781fe868dbb1366e5'}, 'panopticPool': {'id': '0x000000000000305b8621e2475aee38ab5721d525', 'oracleContract': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6

In [ ]:
print('account_funds_per_panoptic_pool')
print(account_funds_per_panoptic_pool)

# Calculate net balances per account by summing NLV and assets across all pools
net_balances = {}
for ppa in account_funds_per_panoptic_pool:
    account_id = ppa['account_id']
    
    # Initialize account balances if not seen before
    if account_id not in net_balances:
        net_balances[account_id] = [Decimal(0), Decimal(0)]
        
    # Add this pool's NLV and assets to running total for this account
    net_balances[account_id][0] += Decimal(ppa['nlv0']) + ppa['assets0']
    net_balances[account_id][1] += Decimal(ppa['nlv1']) + ppa['assets1']

print('net_balances')
print(net_balances)

NameError: name 'account_funds_per_panoptic_pool' is not defined